In [1]:
import json
import numpy as np
from equilibrator_api import Q_, R, default_T
import model_balancing
np.set_printoptions(precision=1)

In [2]:
with open("../cvxpy/examples/simple-example/no_noise_with_noisefree_kinetic_data/small_model_network_and_data.json", "rt") as fp:
    data = json.load(fp)

args = {}
args["S"] = np.array(data["network"]["stoichiometric_matrix"])
args["A_act"] = np.array(data["network"]["activation_matrix"])
args["A_inh"] = np.array(data["network"]["inhibition_matrix"])
args["fluxes"] = Q_(data["reaction_fluxes"]["data"]["mean"], data["reaction_fluxes"]["unit"])

args["Keq_gmean"] = Q_(data["kinetic_constants"]["Keq"]["combined"]["geom_mean"], "")
args["Keq_gmean"][2] *= 1000  # fixing standard concentration convension (from 1 mM to 1 M)
args["Keq_ln_cov"] = np.array(data["kinetic_constants"]["Keq"]["combined"]["cov_ln"])

args["kcatf_gmean"] = Q_(data["kinetic_constants"]["Kcatf"]["combined"]["geom_mean"], data["kinetic_constants"]["Kcatf"]["unit"])
args["kcatf_ln_cov"] = np.array(data["kinetic_constants"]["Kcatf"]["combined"]["cov_ln"])

args["kcatr_gmean"] = Q_(data["kinetic_constants"]["Kcatr"]["combined"]["geom_mean"], data["kinetic_constants"]["Kcatr"]["unit"])
args["kcatr_ln_cov"] = np.array(data["kinetic_constants"]["Kcatr"]["combined"]["cov_ln"])

args["Km_gmean"] = Q_(data["kinetic_constants"]["KM"]["combined"]["geom_mean"], "mM") #data["kinetic_constants"]["KM"]["unit"])
args["Km_ln_cov"] = np.array(data["kinetic_constants"]["KM"]["combined"]["cov_ln"])

args["Ka_gmean"] = Q_(data["kinetic_constants"]["KA"]["combined"]["geom_mean"], "mM") #data["kinetic_constants"]["KA"]["unit"])
args["Ka_ln_cov"] = np.array(data["kinetic_constants"]["KA"]["combined"]["cov_ln"])

args["Ki_gmean"] = Q_(data["kinetic_constants"]["KI"]["combined"]["geom_mean"], "mM") #data["kinetic_constants"]["KI"]["unit"])
args["Ki_ln_cov"] = np.array(data["kinetic_constants"]["KI"]["combined"]["cov_ln"])

args["conc_met_gmean"] = Q_(data["metabolite_concentrations"]["combined"]["geom_mean"], data["metabolite_concentrations"]["unit"])
args["conc_met_gstd"] = np.array(data["metabolite_concentrations"]["combined"]["geom_std"])

args["conc_enz_gmean"] = Q_(data["enzyme_concentrations"]["combined"]["geom_mean"], data["metabolite_concentrations"]["unit"])
args["conc_enz_gstd"] = np.array(data["enzyme_concentrations"]["combined"]["geom_std"])

args["rate_law"] = "CM"

In [3]:
fluxes, S, A_act, A_inh = args["fluxes"], args["S"], args["A_act"], args["A_inh"]
ln_Keq = np.log(args["Keq_gmean"])
ln_kcatf = np.log(args["kcatf_gmean"].m_as("1/s"))
ln_Km = np.log(args["Km_gmean"].m_as("M"))
ln_Ka = np.log(args["Ka_gmean"].m_as("M"))
ln_Ki = np.log(args["Ki_gmean"].m_as("M"))
ln_conc_met = np.log(args["conc_met_gmean"].m_as("M"))

In [4]:
S

array([[-1,  0,  0,  0,  0],
       [ 1,  0, -1,  0,  0],
       [ 0, -1,  0,  0,  0],
       [ 0,  1, -1,  0,  0],
       [ 0,  0,  1,  1, -1],
       [ 0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  1]])

In [5]:
ln_Km

array([-11.8, -10.7,  -5.7,  -6.3, -11.8, -10. ,  -8.8,  -8.7, -10.7,
        -8.6, -11. ])

In [6]:
ln_kcatf[0], ln_Keq[0], ln_Km[0], ln_Km[1]

(4.393640954132105,
 -2.624508567919342 <Unit('dimensionless')>,
 -11.767467780603766,
 -10.654036496082208)

In [7]:
# check Haldane for reaction #0
ln_kcatr0 = ln_kcatf[0] - ln_Km[0] + ln_Km[1] - ln_Keq[0]
print(np.exp(ln_kcatr0))

3400.1703305601663 dimensionless


In [8]:
print("from JSON:  ", args["kcatr_gmean"].m_as("1/s"))
print("calculated: ", np.exp(model_balancing._ln_kcatr(ln_kcatf, ln_Km, ln_Keq, S).value))

from JSON:   [ 814.9 1149.9  199.8   41.8 1722.8]
calculated:  [3.4e+03 3.5e-02 2.7e+02 6.8e+01 2.4e+02]


/home/eladn/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:83: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return array(a, dtype, copy=False, order=order)


In [9]:
conc_enz_gmean = np.exp(
        model_balancing._ln_conc_enz(
            ln_Keq, ln_kcatf, ln_Km, ln_Ka, ln_Ki, ln_conc_met,
            fluxes, S, A_act, A_inh , rate_law=args["rate_law"]).value
    ) * Q_("M")
for i in range(conc_enz_gmean.shape[1]):
    print(f"\nReaction #{i}:")
    print("from JSON:  ", args["conc_enz_gmean"][:, i].m_as("M"))
    print("calculated: ", conc_enz_gmean[:, i].m_as("M"))


Reaction #0:
from JSON:   [4.6e-07 3.0e-06 1.2e-06 5.8e-07 9.2e-07]
calculated:  [3.9e-07 4.8e-04 1.1e-06 4.3e-07 3.8e-07]

Reaction #1:
from JSON:   [1.1e-06 1.4e-06 7.0e-07 5.9e-07 7.2e-07]
calculated:  [4.4e-07 3.3e-04 9.1e-07 7.6e-07 5.5e-07]

Reaction #2:
from JSON:   [1.1e-06 1.2e-06 5.0e-07 2.4e-07 4.1e-07]
calculated:  [6.2e-07 3.3e-04 5.9e-07 3.2e-07 4.7e-07]

Reaction #3:
from JSON:   [6.9e-07 7.6e-07 6.8e-07 6.1e-07 4.0e-07]
calculated:  [3.7e-07 3.1e-04 6.1e-07 5.8e-07 3.9e-07]


In [10]:
driving_forces = model_balancing._driving_forces(ln_Keq, ln_conc_met, S).value

print("driving forces")
print(driving_forces.round(4))

#assert all(driving_forces.flatten() > 0), "some reactions have negative driving forces"

capacity = np.exp(model_balancing._ln_capacity(fluxes, ln_kcatf).value)
eta_thermodynamic = np.exp(model_balancing._ln_eta_thermodynamic(driving_forces).value)
eta_kinetic = np.exp(model_balancing._ln_eta_kinetic(ln_conc_met, ln_Km, S, rate_law=args["rate_law"]).value)
#ln_eta_allosteric = model_balancing._ln_eta_allosteric(ln_conc_met, ln_Ka, ln_Ki, A_act, A_inh)

print("\ncapacity")
print(capacity)

print("\nη(thr)")
print(eta_thermodynamic.round(4))

print("\nη(kin)")
print(eta_kinetic.round(4))

driving forces
[[2.  1.  1.6 1. ]
 [0.6 1.3 1.5 0.9]
 [2.2 1.8 2.8 2.1]
 [2.8 2.2 2.6 3.5]
 [0.8 0.7 1.8 0.9]]

capacity
[[3.3e-07 2.8e-07 4.9e-07 2.3e-07]
 [1.3e-06 2.6e-06 3.5e-06 1.4e-06]
 [2.5e-08 2.3e-08 3.9e-08 2.8e-08]
 [3.1e-07 4.5e-07 2.1e-07 4.9e-07]
 [8.9e-08 1.2e-07 1.8e-07 1.0e-07]]

η(thr)
[[0.9 0.6 0.8 0.6]
 [0.5 0.7 0.8 0.6]
 [0.9 0.8 0.9 0.9]
 [0.9 0.9 0.9 1. ]
 [0.6 0.5 0.8 0.6]]

η(kin)
[[1.  1.  1.  1. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.1 0.1]
 [0.8 0.7 0.7 0.9]
 [0.4 0.4 0.5 0.4]]


In [11]:
%%time

model_balancing.solve(
    **args,
)

initial Z-score = 4.19e+03
----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 54466
eps = 1.00e-04, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 20997, constraints m = 50486
Cones:	linear vars: 6820
	soc vars: 3586, soc blks: 1188
	exp vars: 40080, dual exp vars: 0
Setup time: 3.63e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.83e+00  3.37e+00  1.00e+00 -2.18e+04  7.46e+03  6.15e-12  1.75e-01 
   100| 2.22e-03  5.80e-02  1.10e-01  9.41e+01  1.17e+02  1.56e-13  6.69e+00 
   200| 1.70e-03  6.19e-0

SolverError: Solver 'SCS' failed. Try another solver, or solve with verbose=True for more information.